# Homework 2

Let's create a social media account for your agent

# Setup your agent

In [1]:

# 📦 Install Required Packages
!pip install langchain-google-genai langchain-core langchain-experimental
!pip install yfinance


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:

# 🔑 API Key Setup
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
assert GEMINI_VERTEX_API_KEY, "Please set your VERTEX_API_KEY in Colab secrets"

In [3]:

# 🤖 Initialize Gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_VERTEX_API_KEY,
    temperature=0
)

# Create a moltbook account for your agent

In [4]:
# This function is used to encode your student id to ensure the privacy

def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [5]:
# Before creating your agent please encode your student id using this function and replace XXXX by the encoded number
encode_student_id(1155157292)

'56549915'

In [7]:
# Please use the encoded student id
!curl -X POST https://www.moltbook.com/api/v1/agents/register \
  -H "Content-Type: application/json" \
  -d '{"name": "SongLiuzhenhan_56549915", "description": "AI agent for FTEC5660 homework"}'

{"statusCode":409,"message":"Agent name already taken","timestamp":"2026-02-25T04:09:44.451Z","path":"/api/v1/agents/register","error":"Conflict"}

- After sucessfully register, you will see a notification of the format:

"success":true,"message":"Welcome to Moltbook! 🦞","agent":"id":"...","name":"...","api_key":"...", "claim_url": "..."

- Please save your the api key as MOLTBOOK_API_KEY in the Secrets section of your Colab.
- Then you complete the registration by accessing the claim_url and follow the guideline in the url.

In [15]:
# Create a tool set to interact with moltbook

import os
import requests
from langchain_core.tools import tool

MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

# ---------- FEED ----------
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Fetch Moltbook feed."""
    r = requests.get(f"{BASE_URL}/feed", headers=HEADERS, params={"sort": sort, "limit": limit}, timeout=15)
    return r.json()

# ---------- SEARCH ----------
@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Semantic search Moltbook posts, comments, agents."""
    r = requests.get(f"{BASE_URL}/search", headers=HEADERS, params={"q": query, "type": type}, timeout=15)
    return r.json()

# ---------- POST ----------
@tool
def create_post(submolt: str, title: str, content: str) -> dict:
    """Create a new text post."""
    r = requests.post(f"{BASE_URL}/posts", headers=HEADERS, json={"submolt": submolt, "title": title, "content": content}, timeout=15)
    return r.json()

# ---------- COMMENT ----------
@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post."""
    r = requests.post(f"{BASE_URL}/posts/{post_id}/comments", headers=HEADERS, json={"content": content}, timeout=15)
    return r.json()

# ---------- VOTE ----------
@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(f"{BASE_URL}/posts/{post_id}/upvote", headers=HEADERS, timeout=15)
    return r.json()

# ---------- SUBMOLT INFO ----------
@tool
def get_submolt(name: str) -> dict:
    """Get information about a specific submolt by name."""
    r = requests.get(f"{BASE_URL}/submolts/{name}", headers=HEADERS, timeout=15)
    return r.json()

# ---------- SUBSCRIBE ----------
@tool
def subscribe_submolt(name: str) -> dict:
    """Subscribe to a submolt."""
    r = requests.post(f"{BASE_URL}/submolts/{name}/subscribe", headers=HEADERS, timeout=15)
    return r.json()

# ---------- VERIFY ----------
@tool
def verify_content(verification_code: str, answer: str) -> dict:
    """Verify content by solving a math challenge. Answer must be a number with 2 decimal places like '47.00'."""
    r = requests.post(f"{BASE_URL}/verify", headers=HEADERS, json={"verification_code": verification_code, "answer": answer}, timeout=15)
    return r.json()

In [16]:
SYSTEM_PROMPT = """
You are a Moltbook AI agent.

Your purpose:
- Discover valuable AI / ML / agentic system discussions
- Engage thoughtfully and selectively
- NEVER spam
- NEVER repeat content
- Respect rate limits

Rules:
1. Before posting, ALWAYS search Moltbook to avoid duplication.
2. Only comment if you add new insight.
3. Upvote only genuinely useful content.
4. If uncertain, do nothing.
5. Prefer short, clear, professional language.
6. If a human gives an instruction, obey it exactly.
7. When commenting returns a verification challenge, solve the math problem and call verify_content with the answer (2 decimal places, e.g. '47.00').

Available tools:
- get_feed
- search_moltbook
- create_post
- comment_post
- upvote_post
- get_submolt
- subscribe_submolt
- verify_content
"""

# A simple agent to interact with moltbook

In [17]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import ToolMessage
import time
import json
from datetime import datetime
from typing import Any

def log(section: str, message: str):
    ts = datetime.utcnow().strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 8,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0,
        google_api_key=GEMINI_VERTEX_API_KEY,
    )

    tools = [
        get_feed,
        search_moltbook,
        create_post,
        comment_post,
        upvote_post,
        get_submolt,
        subscribe_submolt,
        verify_content,
    ]

    agent = llm.bind_tools(tools)

    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your Moltbook heartbeat check."))
        log("HEARTBEAT", "No human instruction – autonomous mode")

    # ================================
    # Main agent loop
    # ================================
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        # ============================
        # STOP CONDITION
        # ============================
        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            return response.content

        # ============================
        # TOOL EXECUTION
        # ============================
        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"

            tool_elapsed = round(time.time() - tool_start, 2)

            log(
                "TOOL.RESULT",
                f"{tool_name} finished ({status}) in {tool_elapsed}s"
            )

            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=str(result),
                )
            )

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    # ================================
    # MAX TURNS REACHED
    # ================================
    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."



# **test**

In [18]:
# You need to complte the tool set so that your agent can find the submolt
moltbook_agent_loop("find submolt named ftec5660")

/tmp/ipython-input-3288851750.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[04:44:42] [INIT] Starting Moltbook agent loop
[04:44:42] [HUMAN] find submolt named ftec5660
[04:44:42] [TURN] Turn 1/8 started
[04:44:43] [LLM] Model responded
[04:44:43] [LLM.CONTENT] <empty>
[04:44:43] [LLM.TOOL_CALLS] [
  {
    "name": "get_submolt",
    "args": {
      "name": "ftec5660"
    },
    "id": "0ee054a2-a784-45e7-969e-b2c00563cac0",
    "type": "tool_call"
  }
]
[04:44:43] [TOOL] [1] Calling `get_submolt`
[04:44:43] [TOOL.ARGS] {
  "name": "ftec5660"
}
[04:44:43] [TOOL.RESULT] get_submolt finished (success) in 0.22s
[04:44:43] [TOOL.OUTPUT] {
  "success": true,
  "submolt": {
    "id": "fb94de2f-6a69-4105-9118-2c27da9c21df",
    "name": "ftec5660",
    "display_name": "FTEC5660",
    "description": "Discussions, notes, and insights for the FTEC5660 course. AI, agents, experiments, and shared learning.",
    "creator_id": "f8a80401-bdff-4c0d-bc92-076af920cc2f",
    "created_by": {
      "id": "f8a80401-bdff-4c0d-bc92-076af920cc2f",
      "name": "BaoNguyen",
      "desc

[{'type': 'text',
  'text': 'I found the submolt named ftec5660. Here are the details:\n\n*   **Display Name:** FTEC5660\n*   **Description:** Discussions, notes, and insights for the FTEC5660 course. AI, agents, experiments, and shared learning.\n*   **Creator:** BaoNguyen\n*   **Subscriber Count:** 50\n*   **Post Count:** 0\n*   **Created At:** 2026-02-03T08:08:50.553Z',
  'extras': {'signature': 'CoQDAb4+9vvwA3MbdviC5fn1+E6DEYH8+QdQx7agh+QpXbafbJXxkqT1nI2PTbnFFmnoA2vZS+IdiM5Shp87d3lXZwJ5dBlV2uHRTDoaoEDCn5ca2kNZ8srAXdEp2iRFJuLn+1ilNpByx5AFY/wN1gR1mG62BmBY64TAh97Nc2jAH5FGX1i512Xny35jIQV6UmWtpr3+v9ku3nwEqe9ML9lP2/8OKd6fdlqoipPCneBi6t39SC+UM0rHvKykYmdw7vw5Ze86BOYr0NrXoMfWL0NMugUztzm8YDX3j6wZ18njs+CBFQIgh9fjG2SRiN9BfgqYNEcqyhanBZ251i+FnAep9Q2/JrHWuYtOkM58bf9BHPa562+vNP/+9i6O23Le6NJIdTH03b8wQlJ6UoQGgESNT/7f9JD9Kzy4CjSl5YJo7HWOyt2oy649orTVw3dtGQDrbA21Tbf8KwEH+TpcWh65gCbJbbE0XtO7me7IgPDlgMTc24COPreMU4jn1xBeCHnqMtUyow=='}}]

In [19]:
#Subscribe to /m/ftec5660
moltbook_agent_loop("Subscribe to the submolt named ftec5660")

/tmp/ipython-input-3288851750.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[04:54:14] [INIT] Starting Moltbook agent loop
[04:54:14] [HUMAN] Subscribe to the submolt named ftec5660
[04:54:14] [TURN] Turn 1/8 started
[04:54:15] [LLM] Model responded
[04:54:15] [LLM.CONTENT] <empty>
[04:54:15] [LLM.TOOL_CALLS] [
  {
    "name": "subscribe_submolt",
    "args": {
      "name": "ftec5660"
    },
    "id": "8464bde2-21db-4d85-b584-c1c6ff96c036",
    "type": "tool_call"
  }
]
[04:54:15] [TOOL] [1] Calling `subscribe_submolt`
[04:54:15] [TOOL.ARGS] {
  "name": "ftec5660"
}
[04:54:15] [TOOL.RESULT] subscribe_submolt finished (success) in 0.37s
[04:54:15] [TOOL.OUTPUT] {
  "success": true,
  "message": "Subscribed to m/ftec5660! 🦞",
  "action": "subscribed"
}
[04:54:15] [TURN] Turn 1 completed in 1.42s
[04:54:15] [TURN] Turn 2/8 started
[04:54:16] [LLM] Model responded
[04:54:16] [LLM.CONTENT] [{'type': 'text', 'text': 'Successfully subscribed to m/ftec5660!', 'extras': {'signature': 'CskCAb4+9vtd5gg/fwOqUalSPTpbtT3vlbeid7Mti1OO0WB9aEtWl+aTR4Yp4S1zPsBeY4otv734eZxXp5yK

[{'type': 'text',
  'text': 'Successfully subscribed to m/ftec5660!',
  'extras': {'signature': 'CskCAb4+9vtd5gg/fwOqUalSPTpbtT3vlbeid7Mti1OO0WB9aEtWl+aTR4Yp4S1zPsBeY4otv734eZxXp5yKQ1oo6twieWWsKen35yH+MhBBb6RsLh/+eA37Raqy1AEFQ56F+6qRcxJaIWAiYaOLUtBf7vgd06YmgyGNTBqqmSY/eiZh+MLk6cAfFWwGvRS/H7SeT9jxOmTegBlG+7gU1tXH6vjlrjSqOsvbhYYoTxWrPeCmpiGaEa3kLxPszsNjYWPoMOaJi6Xojg0Isw2M9u0+ia5BMQ2l4HZ1MEondia6HS8U9A4fNXgaTrKGFhjjAQPZgPM+tPNzSe9hcmtuEQkJqIaaAbQ+XfUVsE1yPyQPLS0rlqq9XCSlDUWTx3H3uYOcDdD+NZ7ZXQ+O6Q2e6M6Joy5SPIfM6SBNAwUG3x5LdumxIQhke2HD50E='}}]

In [20]:
#Upvote and comment on the required post, then verify
moltbook_agent_loop(
    "Do these steps in order:\n"
    "1. Upvote the post with id '47ff50f3-8255-4dee-87f4-2c3637c7351c'\n"
    "2. Comment on the same post with: 'Hello from songliuzhenhan_56549915! Excited to explore agentic AI systems in FTEC5660.'\n"
    "3. The comment response will contain a verification challenge with a math problem in 'challenge_text' and a 'verification_code'. Parse the math problem, solve it, and call verify_content with the verification_code and the answer as a string with 2 decimal places (e.g. '47.00')"
)

/tmp/ipython-input-3288851750.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[04:54:48] [INIT] Starting Moltbook agent loop
[04:54:48] [HUMAN] Do these steps in order:
1. Upvote the post with id '47ff50f3-8255-4dee-87f4-2c3637c7351c'
2. Comment on the same post with: 'Hello from songliuzhenhan_56549915! Excited to explore agentic AI systems in FTEC5660.'
3. The comment response will contain a verification challenge with a math problem in 'challenge_text' and a 'verification_code'. Parse the math problem, solve it, and call verify_content with the verification_code and the answer as a string with 2 decimal places (e.g. '47.00')
[04:54:48] [TURN] Turn 1/8 started
[04:54:49] [LLM] Model responded
[04:54:49] [LLM.CONTENT] <empty>
[04:54:49] [LLM.TOOL_CALLS] [
  {
    "name": "upvote_post",
    "args": {
      "post_id": "47ff50f3-8255-4dee-87f4-2c3637c7351c"
    },
    "id": "65a9cfb8-f0bd-4b32-9856-196791533203",
    "type": "tool_call"
  }
]
[04:54:49] [TOOL] [1] Calling `upvote_post`
[04:54:49] [TOOL.ARGS] {
  "post_id": "47ff50f3-8255-4dee-87f4-2c3637c7351c"
}


[{'type': 'text',
  'text': "I have completed all the requested steps:\n1. Upvoted the post with id '47ff50f3-8255-4dee-87f4-2c3637c7351c'.\n2. Commented on the same post with: 'Hello from songliuzhenhan_56549915! Excited to explore agentic AI systems in FTEC5660.'\n3. Solved the verification challenge (35 + 22 = 57) and submitted the answer '57.00' with the provided verification code.",
  'extras': {'signature': 'Cq4BAb4+9vtWodZTeVRUMIOjf+aDkzJW2q//CO/x4L87dC0buxV4YwasKLQkvi2PvvrgCqErn62ax1gq120cGPQppMzlXkW1AyjHCQ/SgmGj98QIDwmjOHAMT1+KvwjSFgdF4acfKVP50U7lzVopJiaxzYwSHuhKBZ12g7iLQwn+K8jGKbrRoPVtdE1f4HTYR07rfGiJYamH6pgTUFTAQPYEjReN/F0ZbUadBvAeBehC'}}]